In [ ]:
from mayavi import mlab
import numpy as np



In [ ]:
%gui qt4

In [ ]:
cd 'C:\Users\amcmorl\Desktop\Head'

In [ ]:
import vtk
from mayavi import mlab, tools
import numpy as np
import mayavi

nasion = np.array([9.5, -2.5, -9])
inion = np.array([0, 6, -14])
left_auricle = np.array([0, -11, -14])
right_auricle = np.array([-12, -2.5, -14])
vertex = (np.mean((nasion, inion, right_auricle,left_auricle), axis=0)) + np.array([0,0,14.5])

ref = np.vstack((nasion, inion, left_auricle, right_auricle, vertex))

v = mlab.figure()

filename = "Head.obj"
 
reader = vtk.vtkOBJReader()
reader.SetFileName(filename)
 
mapper = vtk.vtkPolyDataMapper()
if vtk.VTK_MAJOR_VERSION <= 5:
    mapper.SetInput(reader.GetOutput())
else:
    mapper.SetInputConnection(reader.GetOutputPort())

mapper.SetColorMode(200)
actor = vtk.vtkActor()
actor.SetMapper(mapper)
actor.SetOrientation(180, 0, 0)
actor.GetProperty().SetColor(1,1,1)

tail = np.ones((10,3))
head = np.ones((10,3))

mlab.quiver3d(tail[:,0], tail[:,1], tail[:,2], head[:,0], head[:,1], head[:,2])

mlab.axes(extent=[-10,10,-10,10,-10,10], ranges=[-10,10,-10,10,-10,10], color=(0,0,0))

mlab.points3d(ref[:,0], ref[:,1], ref[:,2], color=(1,0,1), scale_factor=0.5)
# Assign actor to the renderer
v.scene.add_actor(actor)

# Enable user interface interactor
mlab.show()


In [2]:
# First, and before importing any Enthought packages, set the ETS_TOOLKIT
# environment variable to qt4, to tell Traits that we will use Qt.
import os
os.environ['ETS_TOOLKIT'] = 'qt4'
# By default, the PySide binding will be used. If you want the PyQt bindings
# to be used, you need to set the QT_API environment variable to 'pyqt'
#os.environ['QT_API'] = 'pyqt'

# To be able to use PySide or PyQt4 and not run in conflicts with traits,
# we need to import QtGui and QtCore from pyface.qt
from pyface.qt import QtGui, QtCore
# Alternatively, you can bypass this line, but you need to make sure that
# the following lines are executed before the import of PyQT:
#   import sip
#   sip.setapi('QString', 2)

from traits.api import HasTraits, Instance, on_trait_change
from traitsui.api import View, Item
from mayavi.core.ui.api import MayaviScene, MlabSceneModel, \
        SceneEditor
import numpy as np
import vtk
from mayavi import mlab, tools

################################################################################
#The actual visualization
class Visualization(HasTraits):
    scene = Instance(MlabSceneModel, ())

    @on_trait_change('scene.activated')
    def update_plot(self):
        # This function is called when the view is opened. We don't
        # populate the scene when the view is not yet open, as some
        # VTK features require a GLContext.
        
        filename = "Head.obj"
 
        reader = vtk.vtkOBJReader()
        reader.SetFileName(filename)

        mapper = vtk.vtkPolyDataMapper()
        if vtk.VTK_MAJOR_VERSION <= 5:
            mapper.SetInput(reader.GetOutput())
        else:
            mapper.SetInputConnection(reader.GetOutputPort())

        mapper.SetColorMode(200)
        actor = vtk.vtkActor()
        actor.SetMapper(mapper)
        actor.SetOrientation(180, 0, 0)
        actor.GetProperty().SetColor(1,1,1)

     
        self.scene.add_actor(actor)

        # We can do normal mlab calls on the embedded scene.
        #self.scene.mlab.points3d(ref[:,0], ref[:,1], ref[:,2], color=(1,0,1), scale_factor=0.5)
        #self.scene.mlab.quiver3d(tail[:,0], tail[:,1], tail[:,2], head[:,0], head[:,1], head[:,2])
       

    # the layout of the dialog screated
    view = View(Item('scene', editor=SceneEditor(scene_class=MayaviScene),
                     height=250, width=300, show_label=False),
                resizable=True # We need this to resize with the parent widget
                )


################################################################################
# The QWidget containing the visualization, this is pure PyQt4 code.
class MayaviQWidget(QtGui.QWidget):
    def __init__(self, parent=None):
        QtGui.QWidget.__init__(self, parent)
        layout = QtGui.QVBoxLayout(self)
        layout.setContentsMargins(0,0,0,0)
        layout.setSpacing(0)
        self.visualization = Visualization()

        # If you want to debug, beware that you need to remove the Qt
        # input hook.
        #QtCore.pyqtRemoveInputHook()
        #import pdb ; pdb.set_trace()
        #QtCore.pyqtRestoreInputHook()

        # The edit_traits call will generate the widget to embed.
        self.ui = self.visualization.edit_traits(parent=self,
                                                 kind='subpanel').control
        layout.addWidget(self.ui)
        self.ui.setParent(self)


if __name__ == "__main__":
    # Don't create a new QApplication, it would unhook the Events
    # set by Traits on the existing QApplication. Simply use the
    # '.instance()' method to retrieve the existing one.
    app = QtGui.QApplication.instance()
    container = QtGui.QWidget()
    container.setWindowTitle("Embedding Mayavi in a PyQt4 Application")
    # define a "complex" layout to test the behaviour
    layout = QtGui.QGridLayout(container)

    # put some stuff around mayavi
    label_list = []
    for i in range(3):
        for j in range(3):
            if (i==1) and (j==1):continue
            label = QtGui.QLabel(container)
            label.setText("Your QWidget at (%d, %d)" % (i,j))
            label.setAlignment(QtCore.Qt.AlignHCenter|QtCore.Qt.AlignVCenter)
            layout.addWidget(label, i, j)
            label_list.append(label)
    mayavi_widget = MayaviQWidget(container)

    layout.addWidget(mayavi_widget, 1, 1)
    container.show()
    window = QtGui.QMainWindow()
    window.setCentralWidget(container)
    window.show()

    # Start the main event loop.
    app.exec_()

Exception occurred in traits notification handler for object: <mayavi.tools.mlab_scene_model.MlabSceneModel object at 0x000000000F459938>, trait: actor_list_items, old value: <undefined>, new value: <traits.trait_handlers.TraitListEvent object at 0x000000000F3803C8>
Traceback (most recent call last):
  File "C:\Anaconda\lib\site-packages\traits\trait_notifiers.py", line 519, in rebind_call_1
    self.dispatch( getattr( self.object(), self.name ), new )
  File "C:\Anaconda\lib\site-packages\traits\trait_notifiers.py", line 455, in dispatch
    handler( *args )
  File "C:\Anaconda\lib\site-packages\tvtk\pyface\ui\qt4\scene_editor.py", line 197, in _actor_list_items_changed
    event.added)
  File "C:\Anaconda\lib\site-packages\tvtk\pyface\ui\qt4\scene_editor.py", line 210, in _actor_list_changed
    self._add_actors_widgets(new)
  File "C:\Anaconda\lib\site-packages\tvtk\pyface\ui\qt4\scene_editor.py", line 233, in _add_actors_widgets
    actors, widgets = self._separate_actors_widgets(a

Exception occurred in traits notification handler.
Please check the log file for details.


In [ ]:
x = Visualization()
x.scene.mlab.figure()

In [ ]:
nasion = np.array([9.5, -2.5, -9])
inion = np.array([0, 6, -14])
left_auricle = np.array([0, -11, -14])
right_auricle = np.array([-12, -2.5, -14])
vertex = np.mean((np.concatenate(((nasion-inion)[np.newaxis,:], (right_auricle-left_auricle)[np.newaxis,:]), axis=0)), axis=0)

In [ ]:
np.vstack((nasion, inion, left_auricle, right_auricle, vertex))[:,1]

In [ ]:
actor.GetProperty().SetColor(1,0,1)

In [ ]:
def vtk_show(renderer, w=100, h=100):
    """
    Takes vtkRenderer instance and returns an IPython Image with the rendering.
    """
    renderWindow = vtk.vtkRenderWindow()
    renderWindow.SetOffScreenRendering(1)
    renderWindow.AddRenderer(renderer)
    renderWindow.SetSize(w, h)
    renderWindow.Render()

    windowToImageFilter = vtk.vtkWindowToImageFilter()
    windowToImageFilter.SetInput(renderWindow)
    windowToImageFilter.Update()

    writer = vtk.vtkPNGWriter()
    writer.SetWriteToMemory(1)
    writer.SetInputConnection(windowToImageFilter.GetOutputPort())
    writer.Write()
    data = str(buffer(writer.GetResult()))

    from IPython.display import Image
    return Image(data)